# AoC 2016: Day 19
## Part 2

In [ ]:
import pandas as pd
import altair as alt
from itertools import compress, cycle
from collections import deque
from math import log

INPUT = 3014603

Let's build a function to compute the result, for small numbers. This will be useful to gain some intuition for the problem, and build a dataset. I will translate the program quite litteraly, using `collections.deque` to rotate the list, and poping the last element. This would take 5 hours to use this function to compute the result, but it won't be necessary.

In [ ]:
def last_elf_p2(num_elves: int) -> int:
    """Find part 2 for small num_elves."""
    elves = deque(range(1, num_elves + 1))
    while len(elves) > 1:
        middle = len(elves) // 2
        elves.rotate(-middle - 1)
        elves.pop()
        elves.rotate(middle - 1)
    return elves.pop()


last_elf_p2(5)

The function gives the correct answer, so we will build a dataset and plot the result to gain some intuition.

In [ ]:
df = pd.DataFrame({'x':range(1, 500)})
df['y'] = df['x'].apply(last_elf_p2)

In [ ]:
chart = alt.Chart(df).mark_point().encode(
    x='x:Q',
    y='y:Q',
    color='group:N',
    tooltip=['x:Q', 'y:Q', 'group:N']
).interactive()

In [ ]:
chart

We see that for some part of the graph, the slope seems to be `1` and for other, this is `2`. And there are some breakpoints, where the result goes back to one. Let's see where the breakpoints are

In [ ]:
df['diff'] = df['y'] - df['y'].shift(1, fill_value=0)
df['group'] = 'breakpoint'
df.loc[df['diff'] == 1, 'group'] = 'y=x'
df.loc[df['diff'] == 2, 'group'] = 'y=2x'

In [ ]:
chart

What it seems is that the breakpoints are exponentially distributed, and that the slope changes in the middle of the interval, at the same value than the preceding max.

In [ ]:
df.query('group == "breakpoint"')

In [ ]:
df.query('group == "breakpoint"').index.values

In [ ]:
[3**i for i in range(6)]

What is interesting is that the index are the succeding power of 3 : $x_{breakpoint} = 3^n + 1$. (+1 because x is shiftd by one from the index.)

The points where the slope changes are in the middle of the interval : $x_{slope\ change} = 3^n - 3^{n-1} = 2 * 3^{n-1}$.

In [ ]:
df.query('group == "y=2x" and group.shift(1) == "y=x"').index.values

In [ ]:
[2 * 3**i for i in range(1, 6)]

So, we need to figure out where in term of succeding power of 3 we are with our input, and in which half of the interval.

In [ ]:
log(INPUT)/log(3)

In [ ]:
INPUT <= 2 * 3**13

This means that the `INPUT` is in the first part of the curve : $y=x$.

In [ ]:
INPUT - 3**13

And this is the correct answer !

In [ ]:
def last_elf_p2_exact(num_elves: int) -> int:
    n_pow = int(log(num_elves)/log(3))
    if num_elves == 3 **n_pow:
        return num_elves
    elif num_elves < 2 * 3**n_pow:
        return num_elves - 3**n_pow
    else:
        return 2 * num_elves - 3**(n_pow + 1)

In [ ]:
last_elf_p2_exact(INPUT)

In [ ]:
df['check'] = df['x'].apply(last_elf_p2_exact)

In [ ]:
all(df['check'] == df['y'])